In [ ]:
import sim
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import pandas as pd
import random
import numpy as np

def connect(port):
# Establece la conexión a VREP
# port debe coincidir con el puerto de conexión en VREP
# retorna el número de cliente o -1 si no puede establecer conexión
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID

# Conectarse al servidor de VREP
# *** ejecutar cada vez que se reinicia la simulación ***
clientID = connect(19999)

# Identificar dummy
returnCode,dummy=sim.simxGetObjectHandle(clientID,'Dummy',sim.simx_opmode_blocking)
# Identificar las Articulaciones
ret,joint1=sim.simxGetObjectHandle(clientID,'Art_1',sim.simx_opmode_blocking)
ret,joint2=sim.simxGetObjectHandle(clientID,'Art_2',sim.simx_opmode_blocking)

# Alcance mázimo de la pata
returnCode = sim.simxSetJointTargetPosition(clientID, joint1, 0, sim.simx_opmode_oneshot)
returnCode = sim.simxSetJointTargetPosition(clientID, joint2, 0, sim.simx_opmode_oneshot)
returnCode,pos_joint1=sim.simxGetObjectPosition(clientID, joint1, -1, sim.simx_opmode_blocking)
returnCode,pos_dummy=sim.simxGetObjectPosition(clientID, dummy, -1, sim.simx_opmode_blocking)
alcance_pata = pos_joint1[2]-pos_dummy[2]

#------------------------------------------------------------------------------
#
#              INGRESAR VALOR de la altura del robot en centimetros
#
#------------------------------------------------------------------------------
# Altura del robot en relación al piso
altura_robot = 8 # centimetros
altura_robot = altura_robot/100# metros

# Valores para grafica
piso = round((pos_joint1[2]-altura_robot),4)
alcanceMax = round((pos_joint1[2]-alcance_pata),4)



### Generar Datos de Entrenamiento

In [ ]:
# Cantidad de datos generados aleatoriamente
repeticiones = 10000

#[muslo,rodilla,posx,posy,posz]
data_training = np.array([[0,0,0,0,0]])  # Arreglo de características, cada fila representa una instancia

for i in range(repeticiones):
    # Muslo Radiant
    joint1_thetha = random.uniform(-1.5708, 0)
    # Rodilla Radiant
    joint2_thetha = random.uniform(0, 2.0944)
    returnCode = sim.simxSetJointTargetPosition(clientID, joint1, joint1_thetha, sim.simx_opmode_oneshot)
    returnCode = sim.simxSetJointTargetPosition(clientID, joint2, joint2_thetha, sim.simx_opmode_oneshot)
    returnCode,pos_dummy=sim.simxGetObjectPosition(clientID, dummy, -1, sim.simx_opmode_blocking)
    new_row = np.array([joint1_thetha,joint2_thetha,pos_dummy[0],pos_dummy[1],pos_dummy[2]])
    data_training = np.append(data_training, [new_row],axis=0)
    if i == 1:
        data_training = np.delete(data_training, 0, axis=0)

labels = ["Muslo", "Rodilla","Dummy_X", "Dummy_Y", "Dummy_Z"]
data = pd.DataFrame(data_training, columns=labels)

#### Guardar Data de entrenamiento

In [ ]:
# Guardar el FataFrame
# df.to_csv('data.txt', sep='\t', index=False)
# Leer el DataFrame
# df = pd.read_csv('data.txt', sep='\t')
data.to_csv('data.txt', sep='\t', index=False)

#### Graficar Datos

In [ ]:
plt.scatter(data["Dummy_X"], data["Dummy_Z"])
plt.axhline(y=alcanceMax, color='r', linestyle='--')
plt.axhline(y=piso, color='g', linestyle='--')
# Personalizar el gráfico
plt.xlabel('X')
plt.ylabel('Z (Y)')
plt.title('Gráfico de columnas seleccionadas')
# Mostrar el gráfico
plt.show()

#### Encontrar alcances maximos en el piso

In [ ]:
raz_de_piso = data.loc[(data["Dummy_Z"] >= piso*0.9) & (data["Dummy_Z"] <= piso*1.1)]
max_izquierda = [round(raz_de_piso["Dummy_X"].min(), 4) , round(raz_de_piso["Dummy_Z"].loc[raz_de_piso["Dummy_X"].idxmin()], 4)]
max_derecha =   [round(raz_de_piso["Dummy_X"].max(), 4) , round(raz_de_piso["Dummy_Z"].loc[raz_de_piso["Dummy_X"].idxmax()], 4)]
print(max_izquierda, "Alcance maximo adelante")
print(max_derecha, "Alcance maximo trasero")